# Quick Start Guide

This guide assumes that you have already downloaded the pre-processed version of the dataset and unpacked it to `data_path`. Note that this dataset is a selected subset of the entire FACT dataset.

This guide uses PyTorch to run an example model on the data. For Keras & TensorFlow starting points see the files in `FACT/Benchmark`.

In [ ]:
data_path = './data'

We begin by loading the pandas DataFrames of the training, OEIS testing, and synthetic testing data.

In [ ]:
import pandas as pd
import numpy as np
import os

data_training = pd.read_pickle(os.path.join(data_path, 'dataset.pickle'))
data_test_oeis = pd.read_pickle(os.path.join(data_path, 'testset_oeis.pickle'))
data_test_synth = pd.read_pickle(os.path.join(data_path, 'testset_synth.pickle'))


You can now easily inspect the structure of the data with `DataFrame.columns` and `DataFrame.head()`.

In [ ]:
data_training.columns

For the purpose of this quick-start guide we will train a simple transformer without layer normalisation for a classification task. The inputs to our model shall be simply the integer sequences, and the outputs will a vector of beliefs between `0` and `1` that the given sequence belongs to a given class (such as `trigonometric` or `modulo`) or not.

In [ ]:
list_of_sequences = []

for sequence in data_training['sequence']:
	sequence_as_list = [ int(number) for number in sequence.split(',') ]
	list_of_sequences.append(sequence_as_list)

x = np.array(list_of_sequences, dtype=np.float64)
x = x[:,:,np.newaxis]

In [ ]:
y = data_training[['eval_trigonometric', 'eval_polynomial',
       'eval_exponential', 'eval_periodic', 'eval_finite', 'eval_modulo',
       'eval_prime', 'eval_bounded', 'eval_increasing', 'eval_unique']].to_numpy(dtype=np.float64)

For example, the 6th sequence in our dataset is a unique polynomial.

In [ ]:
y[5]

We use a custom wrapper (a sub-class of `torch.utils.data.Dataset`) to serve the dataset through dataloaders to the training loop.

In [ ]:
from data import IntegerDataset
training_dataset = IntegerDataset(x, y)

We will use a transformer encoder complemented by a layer of classification sigmoids. These are described in detail in `transformer.py`.

In [ ]:
import torch
from transformer import TransformerEncoder, EncoderClassifier
import curriculum

encoder = TransformerEncoder(
	num_layers = 4,
	dim_model = 1,
	num_heads = 8,
	dim_feedforward = 256,
	dropout = 0.0,
)
model = EncoderClassifier(encoder, dim_model=50, num_classes=10)

We shall train our model on all training data available for 70 epochs with the AdamW optimizer. Simply run the cell below to start the training and see the intermediate results.

In [ ]:
from collections import namedtuple

meta_config = {
	"wandbosity": 0,
	"verbosity": 2,

	"optimizer": "adamw",
	"lr": 0.009
}
meta_config = namedtuple("meta_config", meta_config.keys())(*meta_config.values())

# make data
# dataloader_kwargs = {'num_workers': 1, 'pin_memory': True} if device=="cuda" else {}
dataloader_kwargs = {}
training_dataloader = torch.utils.data.DataLoader(training_dataset, batch_size=128, shuffle=True, **dataloader_kwargs)

epochs = 70
best_training_loss = float("inf")
for t in range(epochs):
	print(f"Epoch {t+1}\n-------------------------------")
	training_loss = curriculum.train(model, training_dataloader, meta_config)
	if training_loss < best_training_loss:
		torch.save(model.state_dict(), "bestModel.pt")
print("Done!")

model.load_state_dict(torch.load("bestModel.pt"))
model.eval()

That's it! You have now trained a transformer-encoder classifier on the FACT dataset. We hope that this guide will serve you as a starting point for working with FACT.